# Capstone 3: Predicting Customer Lifetime Value (CLV)

## Predicting Customer Lifetime Value (CLV) to Optimize Targeted Marketing Strategies in E-Commerce

This capstone project aims to enhance targeted marketing strategies in the e-commerce sector by accurately predicting CLV for the largest online department store in Brazil, encompassing 100,000 anonymized orders made between 2016 and 2018.

Target feature: `customer_total_spend`

## PRE-PROCESSING, TRAINING, AND MODELING

## Table of Contents
* [Feature Engineering](#feature_engineering)
* [One-Hot Encoding](#one_hot_encoding)
* [Train-Test Split](#train_test_split)
* [Scaling](#scaling)
* [Model Training and Evaluation](#model_training_evaluation)
  * [Baseline Model: Linear Regression](#baseline_linear)
      * [Non-Scaled Data](#nonscaled)
      * [Scaled Data](#scaled)
  * [Alternative Models](#alternative_models)
      * [Random Forest, Gradient Boosting, Lasso](#random_gradient_lasso)
  * [Ensemble Model](#ensemble)
  * [Stacked Model](#stacked)
* [Compare and Select the Best Model](#compare_select)
* [Summary](#summary)

In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import shutil  # Make sure to import the shutil module

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
clv = pd.read_csv('/content/drive/My Drive/Colab Notebooks/df_clv_eda.csv')

In [4]:
clv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92062 entries, 0 to 92061
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_unique_id                   92062 non-null  object 
 1   customer_average_order_value         92062 non-null  float64
 2   customer_tenure_days                 92062 non-null  int64  
 3   customer_recency_days                92062 non-null  int64  
 4   customer_total_orders_frequency      92062 non-null  int64  
 5   customer_total_spend_monetary        92062 non-null  float64
 6   customer_unique_products             92062 non-null  int64  
 7   customer_category_diversity          92062 non-null  int64  
 8   customer_average_product_price       92062 non-null  float64
 9   customer_total_units                 92062 non-null  int64  
 10  customer_average_delivery_time       92062 non-null  float64
 11  customer_average_shipping_co

## Feature Engineering <a class="anchor" id="feature_engineering"></a>

While the dataset above was created from our entire original dataset and was useful for exploring our data in the EDA portion of our analysis, in the context of feature engineering and data pre-processing we know that data leakage occurs when information from outside the training set is used to create or transform features, leading to overly optimistic performance estimates during model evaluation. In short, the statistical properties of the entire dataset, including the test set, were used to create these features, which wouldn't be available in a real-world scenario when making predictions on new data.

For example,
- **Customer Recency and Frequency Metrics (e.g., `customer_recency_days`, `customer_total_orders_frequency`):** These were calculated using data that includes periods extending into what becomes the test set, so there is potential leakage.
- **Average Metrics (e.g., `customer_average_order_value`, `customer_average_product_price`, `customer_average_shipping_cost`):** Similar to scaling, these averages were computed using the entire dataset, including the test set, so the model has seen future information.

Therefore, we need to split our original dataset shown below into train and test subsets first, and then recreate the same features for both subsets.

In [5]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/df_eda.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113367 entries, 0 to 113366
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    113367 non-null  object 
 1   customer_unique_id             113367 non-null  object 
 2   customer_zip_code_prefix       113367 non-null  int64  
 3   customer_city                  113367 non-null  object 
 4   customer_state                 113367 non-null  object 
 5   order_id                       113367 non-null  object 
 6   order_status                   113367 non-null  object 
 7   order_purchase_timestamp       113367 non-null  object 
 8   order_approved_at              113367 non-null  object 
 9   order_delivered_carrier_date   113367 non-null  object 
 10  order_delivered_customer_date  113367 non-null  object 
 11  order_estimated_delivery_date  113367 non-null  object 
 12  order_item_id                 

In [7]:
# Check for duplicates
df.duplicated().sum()

0

We split our original dataset into train and test subsets and recreate the same features on both.

In [8]:
def engineer_customer_features(df):
    # Convert date columns to datetime
    date_columns = [
        'order_purchase_timestamp',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
        'order_approved_at',
        'order_delivered_carrier_date'
    ]
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])

    # Calculate delivery time days
    df['delivery_time_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

    # Determine if deliveries were on time
    df['on_time_delivery'] = df['order_delivered_customer_date'] <= df['order_estimated_delivery_date']

    # Grouping by 'customer_unique_id' and applying multiple aggregations
    aggregations = {
        'order_id': 'nunique',
        'payment_value': 'sum',
        'product_id': 'nunique',
        'product_category': 'nunique',
        'price': 'mean',
        'order_item_id': 'count',
        'delivery_time_days': 'mean',
        'freight_value': 'mean',
        'on_time_delivery': 'mean',
        'payment_installments': 'mean',
        'payment_sequential': 'sum',
        'order_purchase_timestamp': ['min', 'max']
    }

    # Creating initial aggregates
    agg_df = df.groupby('customer_unique_id').agg(aggregations).reset_index()

    # Renaming columns
    agg_df.columns = [
        'customer_unique_id',
        'customer_total_orders',
        'customer_total_spend',
        'customer_unique_products',
        'customer_category_diversity',
        'customer_average_product_price',
        'customer_total_units',
        'customer_average_delivery_time',
        'customer_average_shipping_cost',
        'customer_on_time_delivery_rate',
        'customer_average_installments',
        'customer_total_payment_transactions',
        'first_purchase_date',
        'last_purchase_date'
]

    # Calculating additional features
    agg_df['customer_average_order_value'] = agg_df['customer_total_spend'] / agg_df['customer_total_orders']
    agg_df['customer_tenure_days'] = (agg_df['last_purchase_date'] - agg_df['first_purchase_date']).dt.days
    agg_df['customer_recency_days'] = (agg_df['last_purchase_date'].max() - agg_df['last_purchase_date']).dt.days

    # Preferred Payment Method
    preferred_payment_method = df.groupby(['customer_unique_id', 'payment_type']).size().reset_index(name='counts')
    preferred_payment_method = preferred_payment_method.sort_values('counts', ascending=False).drop_duplicates('customer_unique_id')
    preferred_payment_method = preferred_payment_method[['customer_unique_id', 'payment_type']]
    preferred_payment_method = preferred_payment_method.rename(columns={'payment_type': 'customer_preferred_payment_type'})

    # Merging the preferred payment method
    agg_df = agg_df.merge(preferred_payment_method, on='customer_unique_id', how='left')

    return agg_df

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Feature engineering on both training and test sets
train_features_df = engineer_customer_features(train_df)
test_features_df = engineer_customer_features(test_df)

# Select only the relevant columns
final_columns = [
    'customer_unique_id', 'customer_total_orders', 'customer_total_spend', 'customer_average_order_value',
    'customer_tenure_days', 'customer_recency_days', 'customer_unique_products',
    'customer_category_diversity', 'customer_average_product_price',
    'customer_total_units', 'customer_average_delivery_time',
    'customer_average_shipping_cost', 'customer_on_time_delivery_rate',
    'customer_preferred_payment_type', 'customer_average_installments',
    'customer_total_payment_transactions'
]

train_features_df = train_features_df[final_columns]
test_features_df = test_features_df[final_columns]

In [9]:
train_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76027 entries, 0 to 76026
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_unique_id                   76027 non-null  object 
 1   customer_total_orders                76027 non-null  int64  
 2   customer_total_spend                 76027 non-null  float64
 3   customer_average_order_value         76027 non-null  float64
 4   customer_tenure_days                 76027 non-null  int64  
 5   customer_recency_days                76027 non-null  int64  
 6   customer_unique_products             76027 non-null  int64  
 7   customer_category_diversity          76027 non-null  int64  
 8   customer_average_product_price       76027 non-null  float64
 9   customer_total_units                 76027 non-null  int64  
 10  customer_average_delivery_time       76027 non-null  float64
 11  customer_average_shipping_co

In [10]:
test_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21362 entries, 0 to 21361
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_unique_id                   21362 non-null  object 
 1   customer_total_orders                21362 non-null  int64  
 2   customer_total_spend                 21362 non-null  float64
 3   customer_average_order_value         21362 non-null  float64
 4   customer_tenure_days                 21362 non-null  int64  
 5   customer_recency_days                21362 non-null  int64  
 6   customer_unique_products             21362 non-null  int64  
 7   customer_category_diversity          21362 non-null  int64  
 8   customer_average_product_price       21362 non-null  float64
 9   customer_total_units                 21362 non-null  int64  
 10  customer_average_delivery_time       21362 non-null  float64
 11  customer_average_shipping_co

## One-Hot Encoding <a id="one_hot_encoding"></a>

Next, we apply one-hot encoding, which:
- Fits the encoder on the training subset only to avoid data leakage.
- Transforms both the training and test subsets with the fitted encoder.
- Creates data frames for the encoded features and merges them back into the original data frames after dropping the original column.

In [11]:
# One-hot encoding
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit the encoder only on training data and transform both training and test sets
encoded_train_features = encoder.fit_transform(train_features_df[['customer_preferred_payment_type']])
encoded_test_features = encoder.transform(test_features_df[['customer_preferred_payment_type']])

# Create DataFrame with the encoded features
encoded_columns = encoder.get_feature_names_out(['customer_preferred_payment_type'])
encoded_train_df = pd.DataFrame(encoded_train_features, columns=encoded_columns, index=train_features_df.index)
encoded_test_df = pd.DataFrame(encoded_test_features, columns=encoded_columns, index=test_features_df.index)

# Drop the original 'customer_preferred_payment_type' column and add the encoded features
train_features_df = train_features_df.drop(columns=['customer_preferred_payment_type']).join(encoded_train_df)
test_features_df = test_features_df.drop(columns=['customer_preferred_payment_type']).join(encoded_test_df)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Train-Test Split <a id="train_test_split"></a>

Now we drop non-relevant features. In this case, we only have one.

In [12]:
# Drop non-relevant features
train_features_df = train_features_df.drop(columns=['customer_unique_id'])
test_features_df = test_features_df.drop(columns=['customer_unique_id'])

We split the data (non-scaled) for model evaluation.

In [13]:
# Test-train split
target_column_name = 'customer_total_spend'

X_train = train_features_df.drop(columns=[target_column_name])
y_train = train_features_df[target_column_name]
X_test = test_features_df.drop(columns=[target_column_name])
y_test = test_features_df[target_column_name]

## Scaling <a class="anchor" id="scaling"></a>

Then we scale the data using standardization.

In [14]:
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both train and test data
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

## Model Training and Evaluation <a id="model_training_evaluation"></a>

### Linear Regression <a id="linear regression"></a>

We will use linear regression as our baseline model. We will also use this model to evaluate how the model performs using scaled and non-scaled data.

#### Non-Scaled Data <a id="nonscaled"></a>

First, we evaluate the model using non-scaled data.

In [16]:
# Initialize the linear regression model.
lr = LinearRegression()

# Train and evaluate on non-scaled data
lr.fit(X_train, y_train)
y_pred_non_scaled = lr.predict(X_test)
lr_mse_non_scaled = mean_squared_error(y_test, y_pred_non_scaled)
lr_r2_non_scaled = r2_score(y_test, y_pred_non_scaled)

print("MSE with non-scaled data:", lr_mse_non_scaled)
print("R² with non-scaled data:", lr_r2_non_scaled)


MSE with non-scaled data: 2352.7929843740335
R² with non-scaled data: 0.9821246397014615


#### Scaled Data <a id="scaled"></a>

Then we train and evaluate linear regression on scaled data.

In [17]:
# Train and evaluate on scaled data
lr.fit(scaled_X_train, y_train)
y_pred_scaled = lr.predict(scaled_X_test)
lr_mse_scaled = mean_squared_error(y_test, y_pred_scaled)
lr_r2_scaled = r2_score(y_test, y_pred_scaled)

print("MSE with scaled data:", lr_mse_scaled)
print("R² with scaled data:", lr_r2_scaled)

MSE with scaled data: 2352.7929843740417
R² with scaled data: 0.9821246397014615


We see that the same scores were returned for both non-scaled and scaled data.

Below, we do a final check and confirm that the data was properly scaled.

In [18]:
# Check the mean and standard deviation of the original and scaled data
original_mean = np.mean(X_train, axis=0)
original_std = np.std(X_train, axis=0)

scaled_mean = np.mean(scaled_X_train, axis=0)
scaled_std = np.std(scaled_X_train, axis=0)

print("\nOriginal means:\n", original_mean)
print("Original standard deviations:\n", original_std)

print("\nScaled means (should be close to 0):", scaled_mean)
print("Scaled standard deviations (should be close to 1):", scaled_std)


Original means:
 customer_total_orders                            1.028266
customer_average_order_value                   199.302810
customer_tenure_days                             2.240099
customer_recency_days                          236.570468
customer_unique_products                         1.057782
customer_category_diversity                      1.021374
customer_average_product_price                 125.120254
customer_total_units                             1.192905
customer_average_delivery_time                  12.116452
customer_average_shipping_cost                  20.222332
customer_on_time_delivery_rate                   0.918476
customer_average_installments                    2.911403
customer_total_payment_transactions              1.301906
customer_preferred_payment_type_boleto           0.198364
customer_preferred_payment_type_credit_card      0.753785
customer_preferred_payment_type_debit_card       0.015231
customer_preferred_payment_type_voucher          0.032

We can conclude that, having observed the same scores for both scaled and unscaled data, the original features are on the same scale.

## Alternative Models <a id="alternative_models"></a>

### Random Forest, Gradient Boosting, Lasso <a id="random_gradient_lasso"></a>

We will now evaluate the following models, selected for their diversity:
- Linear Regression (already evaluated) and Lasso are both linear models but with different regularizations.
- Random Forest Regressor and Gradient Boosting Regressor are both tree-based, but Random Forest is bagging while Gradient Boosting is boosting.


In [21]:
# Define the model pipeline using a parameter for switching models
pipeline = Pipeline([
    ('regressor', RandomForestRegressor(random_state=42))  # Placeholder, will be replaced in param_grid
])

# Define a parameter grid with parameters for each of the models
param_grid = [
    {
        'regressor': [RandomForestRegressor(random_state=42)],
        'regressor__n_estimators': [100, 200],
        'regressor__max_depth': [10, 20, None]
    },
    {
        'regressor': [GradientBoostingRegressor(random_state=42)],
        'regressor__n_estimators': [100, 200],
        'regressor__max_depth': [3, 5, 7]
    },
    {
        'regressor': [Lasso(random_state=42)],
        'regressor__alpha': [0.1, 1, 10]
    }
]

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Print best hyperparameters
print(f'Best Hyperparameters: {grid_search.best_params_}')

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print('Best Model:')
print(f' Mean Squared Error: {mse_best}')
print(f' R^2 Score: {r2_best}')

Fitting 3 folds for each of 15 candidates, totalling 45 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Hyperparameters: {'regressor': Lasso(alpha=0.1, random_state=42), 'regressor__alpha': 0.1}
Best Model:
 Mean Squared Error: 2352.6644926868353
 R^2 Score: 0.9821256159179069


In [22]:
# Gathering the results
results = grid_search.cv_results_

# Looping through the results and printing them
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Score: {mean_score:.4f}, Parameters: {params}")

Score: 0.8975, Parameters: {'regressor': RandomForestRegressor(random_state=42), 'regressor__max_depth': 10, 'regressor__n_estimators': 100}
Score: 0.8971, Parameters: {'regressor': RandomForestRegressor(random_state=42), 'regressor__max_depth': 10, 'regressor__n_estimators': 200}
Score: 0.9003, Parameters: {'regressor': RandomForestRegressor(random_state=42), 'regressor__max_depth': 20, 'regressor__n_estimators': 100}
Score: 0.8984, Parameters: {'regressor': RandomForestRegressor(random_state=42), 'regressor__max_depth': 20, 'regressor__n_estimators': 200}
Score: 0.8999, Parameters: {'regressor': RandomForestRegressor(random_state=42), 'regressor__max_depth': None, 'regressor__n_estimators': 100}
Score: 0.9001, Parameters: {'regressor': RandomForestRegressor(random_state=42), 'regressor__max_depth': None, 'regressor__n_estimators': 200}
Score: 0.8798, Parameters: {'regressor': GradientBoostingRegressor(random_state=42), 'regressor__max_depth': 3, 'regressor__n_estimators': 100}
Score:

The Mean Test Score, the average score (e.g., accuracy, mean squared error) across all cross-validation folds for each parameter combination, resulted in the following best alternative models:

- **Random Forest Regressor**: Score ≈ 0.9003
- **Gradient Boosting Regressor**: Score ≈ 0.8942
- **Lasso**: Score ≈ 0.9750

We will use these, along with Linear Regression, for the ensemble and stacked models.

## Ensemble Model <a id="ensemble"></a>

Ensemble models leverage the strength of multiple models to improve overall performance, mitigate individual model weaknesses, and enhance robustness and generalizability.




In [23]:
# Define the ensemble model
ensemble_model = VotingRegressor(estimators=[
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(max_depth=20, n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(max_depth=7, n_estimators=100, random_state=42)),
    ('lasso', Lasso(alpha=0.1, random_state=42))
])

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Evaluate ensemble model performance
ensemble_pred = ensemble_model.predict(X_test)
mse_ensemble = mean_squared_error(y_test, ensemble_pred)
r2_ensemble = r2_score(y_test, ensemble_pred)
print(f'Ensemble MSE: {mse_ensemble}, Ensemble R²: {r2_ensemble}')

Ensemble MSE: 5810.8868198822665, Ensemble R²: 0.955851748857939


## Stacked Model <a id="stacked"></a>

In a stacked model, we combine base models using another model (meta-model) to learn how to best combine the predictions.

In [24]:
# Define base models
base_models = [
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(max_depth=20, n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(max_depth=7, n_estimators=100, random_state=42)),
    ('lasso', Lasso(alpha=0.1, random_state=42))
]

# Define the meta-model (stacker), using a robust model like linear regression for simplicity
stacked_model = StackingRegressor(
    estimators=base_models,
    final_estimator=LinearRegression()
)

# Train the stacked model
stacked_model.fit(X_train, y_train)

# Evaluate stacked model performance
stacked_pred = stacked_model.predict(X_test)
mse_stacked = mean_squared_error(y_test, stacked_pred)
r2_stacked = r2_score(y_test, stacked_pred)
print(f'Stacked MSE: {mse_stacked}, Stacked R²: {r2_stacked}')

Stacked MSE: 2403.328311494225, Stacked R²: 0.9817406972186007


## Compare and Select the Best Model <a id="compare_select"></a>

Here are the best final scores of all of the models we evaluated:

- **Linear Regression**: MSE = 2352.7929843740335,
R² = 0.9821246397014615
- **Alternative Models: Lasso**: MSE = 2352.6644926868353,
 R² = 0.9821256159179069
- **Ensemble**: MSE = 5810.8868198822665, R² = 0.955851748857939
- **Stacked**: MSE = 2403.328311494225, R² = 0.9817406972186007

The Lasso model gave the best performance indicating that it is the best choice for overall predictive accuracy.

Linear regression also performed well but was slightly outperformed.


## Summary <a id="summary"></a>

In this capstone project, the goal was to improve targeted marketing strategies in a Brazilian e-commerce setting by accurately predicting Customer Lifetime Value (CLV) using a dataset of 100,000 anonymized orders from 2016 to 2018. The key metric for prediction was `customer_total_spend`.

Here we used pre-processing, training, and modeling to make a decision on the model that would have the best predictive power.

Initially, in the data wrangling portion of our capstone, we created some dervied features using the entire dataset. However, we know that data leakage occurs when information from outside the training set is used to create or transform features, leading to overly optimistic performance estimates during model evaluation. Therefore, to prevent data leakage, we first took the original dataset and divided it into training and test subsets. Features were then recreated on each subset, ensuring a realistic scenario for predicting new data.

One-hot encoding then fit the encoder on the training subset and transformed both the training and test subsets. The encoded features were merged back into the original datasets after dropping the original categorical columns.

The data (excluding non-relevant features) was split into training and test sets for modeling, and standardization was fit on the training set and transformed both the training and test sets to scale the data.

The modeling began with linear regression, which served as the baseline model. Evaluation was done on both scaled and non-scaled data, revealing identical performance, confirming the original features were on a consistent scale.

Alternative models were also explored, including Random Forest, Gradient Boosting, and Lasso, which were chosen for their diverse approaches: Linear Regression (already evaluated) and Lasso are both linear models but with different regularizations. Random Forest Regressor and Gradient Boosting Regressor are both tree-based, but Random Forest is bagging while Gradient Boosting is boosting.

Ensemble modeling aggregated predictions through voting, while stacked modeling used a meta-model to combine base model predictions.

Comparing all final models, the Lasso model emerged as the most accurate, with a Mean Squared Error (MSE) of 2352.66 and an \(R²\) of 0.9821, closely followed by Linear Regression.

In conclusion, the Lasso model provided the best predictive performance, marking it as the optimal model for predicting Customer Lifetime Value in this e-commerce context.